In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('../../')

In [3]:
from programs import lstm_tool

In [4]:
df = pd.read_excel('./documents/nov_nine_var.xlsx').iloc[:,1::].to_numpy()

scaler = MinMaxScaler()
scale_df = scaler.fit_transform(df[:,0:21])

In [5]:
scale_df.dtype

dtype('float64')

In [6]:
x, y = lstm_tool.make_sequence(scale_df, 60)

In [7]:
x.dtype

dtype('float64')

In [8]:
pp_input = tf.keras.layers.Input(shape=(60, 21))
pp_lstm_layer0 = tf.keras.layers.LSTM(64, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True, dropout=0.2)(pp_input)
pp_lstm_layer1 = tf.keras.layers.LSTM(32, activation='tanh', return_sequences=False)(pp_lstm_layer0)
pp_dense_layer = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(pp_lstm_layer1)
pp_output_layer = tf.keras.layers.Dense(units=21, activation='linear')(pp_dense_layer)

pp = tf.keras.Model(pp_input, pp_output_layer)

In [9]:
lstm_input = tf.keras.layers.Input(shape=(60, 21))

input_lstm_layer = tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True)(lstm_input)
hidden_lstm_layer0 = tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=False)(input_lstm_layer)
dropout_lstm_layer = tf.keras.layers.Dropout(0.2)(hidden_lstm_layer0)
hidden_lstm_layer1 = tf.keras.layers.Dense(128, activation='relu')(dropout_lstm_layer)
hidden_lstm_layer2 = tf.keras.layers.Dense(128, activation='relu')(hidden_lstm_layer1)

lstm_layer = tf.keras.models.Model(lstm_input, hidden_lstm_layer2)

state_input = tf.keras.layers.Input(shape=(21,))

input_state_layer = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(state_input)
dropout_state_layer = tf.keras.layers.Dropout(0.3)(input_state_layer)
hidden_state_layer0 = tf.keras.layers.Dense(64, activation='relu')(dropout_state_layer)
hidden_state_layer1 = tf.keras.layers.Dense(64, activation='relu')(hidden_state_layer0)

state_layer = tf.keras.models.Model(state_input, hidden_state_layer1)

input_concat_layer = tf.keras.layers.concatenate([lstm_layer.output, state_layer.output])
hidden_concat_layer0 = tf.keras.layers.Dense(32, activation='relu')(input_concat_layer)
dropout_concat_layer = tf.keras.layers.Dropout(0.3)(hidden_concat_layer0)
hidden_concat_layer1 = tf.keras.layers.Dense(32, activation='relu')(dropout_concat_layer)
output_concat_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_concat_layer1)

sp = tf.keras.models.Model([lstm_input, state_input], output_concat_layer)

In [10]:
pp_optim = tf.keras.optimizers.Adam(1e-4)
sp_optim = tf.keras.optimizers.Adam(1e-4)

In [13]:
def pp_loss(p_):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(p_), p_)

def sp_loss(r, p):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(r), r) + tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(p), p)

In [14]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as pp_tape, tf.GradientTape() as sp_tape:
        predicted_x = pp(x, training=True)

        real_output = sp([x, y], training=True)
        fake_output = pp([x, predicted_x], training=True)

        p_loss = pp_loss(fake_output)
        s_loss = sp_loss(real_output, fake_output)
    
    pp_gd = pp_tape.gradient(p_loss, pp.trainable_variables)
    sp_gd = sp_tape.gradient(s_loss, sp.trainable_variables)

    pp_optim.apply_gradients(zip(pp_gd, pp.trainable_weights))
    sp_optim.apply_gradients(zip(sp_gd, sp.trainable_weights))

In [ ]:
def train(dataset, epochs):
    inputs = [d for d in dataset[::2]]
    outputs = [d for d  in dataset[1:2]]

    for epoch in range(epochs):
        for i, o in zip(inputs, outputs):
            train_step(i, o)